In [ ]:
### This cell contains functions for drawing Bloch spheres used later in the notebook ###
### Please ignore and proceed to the next cell ###

def bloch_vector(*args):
  from numpy import cos, sin, mgrid, pi, arccos, arcsin, array
  from matplotlib.pyplot import figure, axis, show, plot, tight_layout, margins
  
  #convert amplitudes to angles
  angles = []
  for amplitudes in args:
    a0, a1 = amplitudes
    theta = 2*arccos(np.abs(a0))
    phi = np.angle((np.conjugate(a0)*a1)/(np.abs(a0)*np.abs(a1)))
    angles.append((theta, phi))

  fig = figure(figsize=(8,8))
  ax = fig.add_subplot(projection='3d')

  #draw Bloch sphere grid
  u, v = mgrid[0:2 * pi:40j, 0:2 * pi:40j]
  x1 = cos(u) * sin(v)
  y1 = sin(u) * sin(v)
  z1 = cos(v)
  ax.plot_wireframe(x1, y1, z1, color="0.5", linewidth=0.1)

  #add internal axes and labels
  ax.plot([-1, 1], [0, 0], [0,0], c='k')
  ax.plot([0,0], [0, 0], [-1, 1], c='k')
  ax.plot([0, 0], [-1, 1], [0,0], c='k')
  ax.text(1.1, 0, 0, r"$x$", size='xx-large')
  ax.text(0, 1.1, 0, r"$y$", size='xx-large')
  ax.text(0, 0, 1.1, r"$|0\rangle$", size='xx-large')
  ax.text(0, 0, -1.1, r"$|1\rangle$", size='xx-large')

  #plot state vectors
  for vector in angles:
    theta, phi = vector
    x, y, z = (sin(theta)*cos(phi), sin(theta)*sin(phi), cos(theta))
    plot([0, sin(theta)*cos(phi)], [0, sin(theta)*sin(phi)], [0, cos(theta)], lw=3)
    plot([sin(theta)*cos(phi), sin(theta)*cos(phi)], [sin(theta)*sin(phi), sin(theta)*sin(phi)], [0, cos(theta)], 'k:')
    plot([0, sin(theta)*cos(phi)], [0, sin(theta)*sin(phi)], [0, 0], 'k:')
  
  axis('off')
  show()

def bloch_ring(*args):
  from numpy import log, cos, sin, mgrid, pi, arccos, array, exp, linspace
  from matplotlib.pyplot import figure, axis, show, plot, tight_layout, margins

  #convert amplitudes to angles
  zs = []
  rings = []
  for probability in args:
    theta = 2*arccos(probability**.5)
    zs.append(cos(theta))
    rings.append(sin(theta)*exp(1j*linspace(0, 2*np.pi, 40)))

  fig = figure(figsize=(8,8))
  ax = fig.add_subplot(projection='3d')

  #draw Bloch sphere grid
  u, v = mgrid[0:2 * pi:40j, 0:2 * pi:40j]
  x1 = cos(u) * sin(v)
  y1 = sin(u) * sin(v)
  z1 = cos(v)
  ax.plot_wireframe(x1, y1, z1, color="0.5", linewidth=0.1)

  #add internal axes and labels
  ax.plot([-1, 1], [0, 0], [0,0], c='k')
  ax.plot([0,0], [0, 0], [-1, 1], c='k')
  ax.plot([0, 0], [-1, 1], [0,0], c='k')
  ax.text(1.1, 0, 0, r"$x$", size='xx-large')
  ax.text(0, 1.1, 0, r"$y$", size='xx-large')
  ax.text(0, 0, 1.1, r"$|0\rangle$", size='xx-large')
  ax.text(0, 0, -1.1, r"$|1\rangle$", size='xx-large')

  #plot state vectors
  for z, ring in zip(zs, rings):
    plot(ring.real, ring.imag, z, '-', lw=3)
  
  axis('off')
  show()

$\newcommand{\ket}[1]{\left|{#1}\right\rangle}$
$\newcommand{\bra}[1]{\left\langle{#1}\right|}$
$\newcommand{\expec}[3]{\left\langle{#1}\left|{#2}\right|{#3}\right\rangle}$
$\newcommand{\braket}[2]{\left\langle{#1}|{#2}\right\rangle}$
$\newcommand{\op}[2]{\left|{#1}\right\rangle\left\langle{#2}\right|}$
$\newcommand{\X}{\mathrm{X}}$
$\newcommand{\Y}{\mathrm{Y}}$
$\newcommand{\Z}{\mathrm{Z}}$
$\newcommand{\H}{\mathrm{H}}$
$\newcommand{\CX}{\mathrm{CNOT}}$
$\newcommand{\RX}{\mathrm{RX}}$
$\newcommand{\RY}{\mathrm{RY}}$
$\newcommand{\RZ}{\mathrm{RZ}}$

# 1. myQLM Basics

## 1.1 Creating and Executing a Quantum Program

Let's start with a simple example to see how we create a quantum program and initialise a register of qubits. We'll create a single qubit register and try preparing an equal superposition of the computational basis states,

$$
\ket{\psi} = \frac{1}{\sqrt{2}} \ket{0} + \frac{1}{\sqrt{2}} \ket{1}
$$

Qubits are always initalised in the state $\ket{0}$, so we can prepare this superposition by applying the Hadamard gate to the intial qubit state. When measured in the computational basis (known as a $z$-axis measurement), an equal superposition should return the results $\ket{0}$ and $\ket{1}$ with equal probability.

Creating and executing a quantum program within the QLM framework requires the following steps:

1. Initialise an empty program and allocate a register of qubits
2. Add gates to the program and build a quantum circuit
3. Convert the circuit to a job (incl. measurement) and submit it to a quantum processing unit

In [ ]:
from qat.lang.AQASM import Program, H
from qat.lang.AQASM.bits import Qbit

prog = Program() #create an empty program
qbits = prog.qalloc(1) #allocate one qubit to the register

H(qbits[0]) #apply a Hadamard gate to qubit 0
circ = prog.to_circ() #convert the program to a quantum circuit

#display the circuit
print("Quantum circuit:")
%qatdisplay circ

Quantum circuit:


Now we can convert our circuit to a job and submit the job to a quantum processing unit (QPU)

Try running the following cell a few times, and see how the measurement result varies with each run, according to the probabilistic nature of quantum mechanics:

In [ ]:
from qat.qpus import get_default_qpu #import the default QPU
qpu = get_default_qpu()

job = circ.to_job(nbshots=1) #convert circuit to executable job
result = qpu.submit(job) #submit the job to a QPU

print("Measured state: ", result[0].state) #print the state measured

To perform a measurement using myQLM we need to choose a quantum processing unit (QPU). In the example above we used the default QPU, which is a noiseless numpy-based simulator. The Atos commercial QLM provides other QPUs that simulate the noise and architecture of real quantum computers.

## 1.2 Performing a Measurement

Let's try preparing a qubit in some unknown superposition by rotating the state vector by a random amount about the x-axis. Then we'll perform a measurement to try and determine that state of the qubit empirically.

For an unknown state,

$$
\ket{\psi} = \cos\left(\theta/2\right) \ket{0} + e^{i\phi}\sin\left(\theta/2\right) \ket{1}
$$

we need to repeat a large number of measurements in order accurately estimate the probability amplitude of each basis state in the superposition,

$$
p(\ket{0}) = |\braket{0}{\psi}|^2 = |\cos\left(\theta/2\right)|^2; \qquad p(\ket{1}) = |\braket{1}{\psi}|^2 = |\sin\left(\theta/2\right)|^2
$$

A $z$-axis measurement can't recover the relative phase $e^{i\phi}$, but we can use the probability estimate from repeated measurements to plot a ring representing the locus of possible state vectors.

In [ ]:
from qat.lang.AQASM import RY, RX
import numpy as np

def XY_rotation(qbits, x_angle, y_angle):
  RX(x_angle)(qbits[0]) #rotate about the x-axis by a random amount
  RY(y_angle)(qbits[0]) #rotate about the y-axis by a random amount
  return

np.random.seed(42)
x_angle, y_angle = np.random.random(size=(2,))*np.pi

prog = Program() #create an empty program
qbits = prog.qalloc(1) #allocate one qubit to the register
XY_rotation(qbits, x_angle, y_angle)
circ = prog.to_circ()

Within the myQLM framework we can execute multiple runs of the circuit (and multiple measurements), using the `nbshots` parameter when creating a job. After submitting the job we get back a `Result` object, which contains a `Sample` for each state measured. The sample contains a `state` label and a `probability` estimate for that state. The probability represents the number of times the state is measured out of the total `nbshots`.

Try varying the number of times we repeat the measurement by adjusting the `n_measurements` variable, and see how the estimated probabilities converge for an increasing number of samples.

In [ ]:
n_measurements = 10 #number of circuit runs/measurements

job = circ.to_job(nbshots=n_measurements) #convert circuit to executable job
result = qpu.submit(job) #submit the job to a QPU

print("Probability estimate:")
for sample in result: #print the estimated probability of each state
  print(f" State {sample.state} = {sample.probability}")

bloch_ring(*[sample.probability for sample in result if sample.state.value==('0',)]) #plot a locus of possible state vectors

## 1.3 Sampling State Amplitudes

The advantage of developing algorithms using simulators, rather than actual quantum computers, is that we can in fact examine the state amplitudes and inspect the phase of the state vector.

Using myQLM we can do this by calling `to_job` without specifying the number of shots (equivalent to `nbshots = 0`). 

In [ ]:
job = circ.to_job() #create job with zero shots
result = qpu.submit(job)

print("Amplitudes:")
for sample in result: #print the complex amplitude of each state
  print(f" State {sample.state} = {sample.amplitude}")

bloch_vector([sample.amplitude for sample in result]) #plot state vector

## 1.4 Mini Task: Measuring in Other Bases

Experimentally determining the phase $\phi$ means we have to obtain the state of the qubit in the $x$ and $y$ axes - corresponding to measurements in the $\ket{\pm}$ and $\ket{\pm i}$ bases respectively,

$$
\ket{\pm} = \frac{\ket{0} \pm \ket{1}}{\sqrt{2}}; \qquad \ket{\pm i} \frac{\ket{0} \pm i\ket{1}}{\sqrt{2}}
$$

Unfortunately, most quantum computers only implement measurements in the $z$ basis, so we have obtain the probabilities $|\braket{\pm}{\psi}|^2$ and $|\braket{\pm i}{\psi}|^2$ via a measurement in the computational basis. We can do this by performing a rotation that is equivalent to a basis transformation, i.e one that rotates the $x$ and $y$ axes onto the $z$ axis.

**Using the code cell below, initialise two programs each with a single qubit register. Then using $x$ and $y$ rotation gates, construct a pair of circuits that allows us to determine the phase $\phi$ of an arbitrary quantum state.**

In [ ]:
#################################
### Create your programs here ###
#################################

### Hint

See below for a hint


Measuring an arbitrary qubit state in the $x$ and $y$ bases yields the results $\ket{+}$ and $\ket{+i}$ with probability

\begin{align}
p(\ket{+}) = |\braket{+}{\psi}|^2 &= \frac{1}{2} + 2\cos\phi \sin\theta \\
p(\ket{+i}) = |\braket{+i}{\psi}|^2 &= \frac{1}{2} + 2\sin\phi \sin\theta
\end{align}

Which allows us to write

$$
\tan\phi = \frac{p(\ket{+i}) - \frac{1}{2}}{p(\ket{+}) - \frac{1}{2}}
$$

### Solution

See below for an example solution

In [ ]:
#x-axis measurement
x_prog = Program()
x_qbits = x_prog.qalloc(1)
XY_rotation(x_qbits, x_angle, y_angle)

RY(-np.pi/2)(x_qbits[0]) #map |+> |-> to |0> |1>

x_meas = x_prog.to_circ()
job = x_meas.to_job()
result = qpu.submit(job)
plus_prob = result[0].probability

#y-axis measurement
y_prog = Program()
y_qbits = y_prog.qalloc(1)
XY_rotation(y_qbits, x_angle, y_angle)

RX(np.pi/2)(y_qbits[0]) #map |+i> |-i> to |0> |1>

y_meas = prog.to_circ()
job = y_meas.to_job()
result = qpu.submit(job)
iplus_prob = result[0].probability
print(x_angle, y_angle)
print(plus_prob, iplus_prob)

#calculate phase angle
phi = np.arctan((iplus_prob - .5)/(plus_prob - .5))
print(f"The phase angle is: {phi/np.pi:.2f}π radians")

## 1.5 Expectation Value of an Observable

We can construct an arbitrary measurement with myQLM using the 'observable' mode for measurements. This measurement mode takes care of the basis transformation for us and returns the expectation value $\left\langle O \right\rangle$ of the observable

$$
\left\langle O \right\rangle = \expec{\psi}{O}{\psi}
$$

An observable can be specified either as a sum of Pauli matrix terms, or directly in matrix form as a `numpy.ndarray`. Examples of each construction method are shown below, where we calculate the expectation values of the Pauli $\X$ and $\Y$ operators in order to obtain the phase angle $\phi$.

In [ ]:
from qat.core import Observable, Term

prog = Program()
qbits = prog.qalloc(1)
XY_rotation(qbits, x_angle, y_angle)

rand_circ = prog.to_circ()

#construct each observable
x_obs = Observable(1, pauli_terms=[Term(1, "X", [0])]) #construction of X using Pauli terms
y_obs = Observable(1, matrix=np.array([[0, -1j],[1j,  0]])) #construction of Y using direct matrix representation

expec = [None, None] #store the expectation value of each observable
for i, O in enumerate([x_obs, y_obs]): #run the circuit and perform a measurement of each observable
  job = rand_circ.to_job(observable=O)
  result = qpu.submit(job)
  expec[i] = result.value
phi = np.arctan(expec[1]/expec[0])

print(f"The phase angle is: {phi/np.pi:.2f}π radians")

# 2. Entangled States

## 2.1 Bell States

We'll try to prepare a maximally entangled, so-called Bell state of the two-qubit states $\left|00\right\rangle$ and $\left|11\right\rangle$ using a Hadamard gate, followed by a CNOT gate. 

The register starts in the two-qubit state $\ket{00}$, and applying the Hadamard to the zeroth qubit we prepare the following superposition

$$
(\mathrm{H}\otimes\mathrm{I}) \ket{00} = \mathrm{H}\ket{0} \otimes \mathrm{I}\ket{0} = \left(\frac{ \ket{0} + \ket{1}}{\sqrt{2}} \right) \otimes \ket{0} = \frac{\ket{00} + \ket{10}}{\sqrt{2}}
$$

The CNOT gate flips the state of the target bit if the control bit is in the state $\ket{1}$, and so applying the CNOT gate to the superposition above

$$
\mathrm{CNOT}\left(\frac{\ket{00} + \ket{10}}{\sqrt{2}}\right) = \frac{\mathrm{CNOT}\ket{00} + \mathrm{CNOT}\ket{10}}{\sqrt{2}} = \frac{\ket{00} + \ket{11}}{\sqrt{2}} = \ket{\Phi^+}
$$

In [ ]:
from qat.lang.AQASM import CNOT

def create_bell_pair(qbits):
  H(qbits[0]) #apply a Hadamard gate to qubit 0
  CNOT(qbits[0], qbits[1]) #apply a CNOT gate to qubit 1 with the qubit 0 as the control
  return

prog = Program()
qbits = prog.qalloc(2)

create_bell_pair(qbits)

circ = prog.to_circ()
job = circ.to_job()
result = qpu.submit(job)

%qatdisplay circ
for sample in result:
  print(sample.amplitude, sample.state) #print each complex amplitude and basis vectore

## 2.2 Superdense Coding

We can use this maximally entangled state to demonstrate a protocol known as superdense coding - a quantum concept that allows one party (Alice) to send two classical bits of information to a second party (Bob) by sending only a single qubit. The idea is for Alice to use her half of the entangled system to manipulate her qubit and Bob's qubit simultaneously in such a way that when the two qubits are disentangled, each qubit can be measured in either the state $\ket{0}$ or $\ket{1}$. She can then send her qubit to Bob, who performs the disentangling, and then measures the state of both qubits.

After preparing the Bell state $\ket{\Phi^+}$, one of the qubits is given to Alice and the second qubit is given to Bob. Now imagine Alice travels to some remote location, taking her qubit with her. From this remote location Alice operates on her qubit to transform the entangled state to any of the four Bell states: $\ket{\Phi^+}$, $\ket{\Phi^-}$, $\ket{\Psi^+}$, $\ket{\Psi^+}$. The encoding scheme is implemented below:

In [ ]:
from qat.lang.AQASM import X, Y, Z

def encode_message(qbits, msg):
  if len(msg) != 2 or not set(msg).issubset({"0", "1"}):
    raise ValueError(f"message '{msg}' is invalid")
  if msg[1] == "1":
    X(qbits[0])
  if msg[0] == "1":
    Z(qbits[0])

After encoding the two bits using her half of the bipartite system, Alice can send her qubit to Bob using a quantum network - in the case of photon polarization qubits, this could be a [fibre network](https://physicsworld.com/a/multi-user-communication-network-paves-the-way-towards-the-quantum-internet/). 

Bob can then decode the two classical bits by applying a CNOT (with Alice's qubit as the control), and a Hadamard on Alice's qubit - this is the reverse of the computation used to create the original Bell state, and is equivalent to projecting the entangled state onto the computational basis.

$$
\begin{matrix}
\text{Initial State} \\
\ket{\Phi^+} \\
\ket{\Phi^+} \\
\ket{\Phi^+} \\
\ket{\Phi^+} \\
\end{matrix}
\rightarrow
\begin{matrix}
\text{Alice's Operation} \\
\mathrm{I} \\
\X \\
\Z \\
\Z\X \\
\end{matrix}
\rightarrow
\begin{matrix}
\text{Initial State} \\
\ket{\Phi^+} \\
\ket{\Psi^+} \\
\ket{\Phi^-} \\
\ket{\Psi^-} \\
\end{matrix}
\rightarrow
\begin{matrix}
\text{Decodes To} \\
\ket{00} \\
\ket{01} \\
\ket{10} \\
\ket{11} \\
\end{matrix}
$$

In [ ]:
def decode(qbits):
  CNOT(qbits[0], qbits[1])
  H(qbits[0])
  return prog

In this way Alice sends only a single qubit, but is able to encode two classical bits by altering the state of her qubit and the state of Bob's qubit simultaneously (via their entanglement) before sending her qubit.

You can comment out the `decode` function below to see how the encoding scheme prepares different Bell states.

In [ ]:
prog = Program()
qbits = prog.qalloc(2)

create_bell_pair(qbits) #create the initial Bell pair
# Alice and Bob now take one qubit each
encode_message(qbits, "11") #Alice encodes a message using her qubit
# Alice sends her qubit to Bob
decode(qbits) #Bob decodes the two bit message

circ = prog.to_circ() #convert the program to a quantum circuit
job = circ.to_job() #convert circuit to executable job
result = qpu.submit(job) #submit the job to a QPU

%qatdisplay circ
print("\nBob measures the state:")
for sample in result:
  print(f"  {sample.state} with probability {abs(sample.amplitude)**2:3.2f}") #print each complex amplitude and basis vector

# 3. Encoding Schemes
The example of superdense coding introduced above is an example of so-called 'basis encoding'. The 'encoding block' in that example is made up of the gates that Alice executes before sending her qubit to Bob, where each possible binary message is encoded in one of the Bell states.

One of the most important aspects of designing a quantum algorithm is the method used to encode classical data in a register of qubits. Encoding methods broadly fall into three categories:

1. Basis encoding
2. Angle encoding
3. Amplitude encoding

## 3.1 Basis Encoding

In general, basis encoding works by representing each input value with a basis state. In the superdense coding example this is the basis of Bell states, but in general it can be any orthonormal basis set - usually the computational basis.

Basis encoding schemes can be as simple as encoding integer real numbers via their binary representation, i.e

\begin{align}
0 &\rightarrow \ket{000} \\
1 &\rightarrow \ket{001} \\
2 &\rightarrow \ket{010} \\
3 &\rightarrow \ket{011} \\
& \vdots \\
7 &\rightarrow \ket{111} \\
\end{align}

or more complicated, where database entries are associated abstractly to a particular binary string.

For a system of $N$ qubits, there are $2^N$ basis states, and so encoding $M$ distinct inputs requires $N \geq \log_2 M$ qubits.

## 3.2 Angle Encoding

Angle encoding uses parameterised gates to encode input values in the the angle of a qubit vector on the Bloch sphere. Typically this is achieved by normalising the input values in the range $[-\pi, \pi]$ and applying an $R_x(\theta)$ or $R_y(\theta)$ rotation, or alternatively by applying the $H$ gate followed by a $R_z(\theta)$ rotation.



The advantage of angle encoding is that it allows us to encode floating point data straightforwardly, since the rotation angle $\theta$ is a continous variable. 

However, since we can only encode a single variable in each qubit we need $N \geq M$ qubits to encode $M$ distinct inputs. Dense angle encoding, where we use an $R_x$ rotation followed by an $R_z$ rotation to encode two features per qubit allows us to achieve an encoding density of $N \geq M/2$.

![image](https://drive.google.com/uc?export=view&id=1dIHwygv6Il0inz6994imq4V59ro6YUv_)

## 3.3 Amplitude Encoding

Amplitude encoding embeds the data in the amplitude of computational basis states through the following transformation

$$
S_x \ket{0}^{\otimes n} = \frac{1}{\|\vec{x}\|} \sum_{i=1}^{2^n} x_i \ket{i}
$$

Amplitude encoding allows us to embed $M$ data features using $N \geq log_2 M$ qubits, much like the basis embedding scheme. It also allows us to embed floating point data much like the angle embedding scheme.

In a way, amplitude encoding is the holy grail of quantum embedding. The problem is that in practice, realising the circuit that applies $S_x$ is very challenging and generally requires a large number of gates, increasing the circuit depth and computational complexity. 